In [4]:
import pandas as pd

# URL of the raw CSV file
url = "D:\projects\mop_hackathon\data.csv"

# Read the CSV file directly into a DataFrame
df = pd.read_csv(url)

# Display the DataFrame
df.head()


,Datetime,AEP_MW
0,31-12-2004 01:00,13478
1,31-12-2004 02:00,12865
2,31-12-2004 03:00,12577
3,31-12-2004 04:00,12517
4,31-12-2004 05:00,12670


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
%matplotlib inline

In [6]:
dataset = df
dataset["Month"] = pd.to_datetime(df["Datetime"]).dt.month
dataset["Year"] = pd.to_datetime(df["Datetime"]).dt.year
dataset["Date"] = pd.to_datetime(df["Datetime"]).dt.date
dataset["Time"] = pd.to_datetime(df["Datetime"]).dt.time
dataset["Week"] = pd.to_datetime(df["Datetime"]).dt.week
dataset["Day"] = pd.to_datetime(df["Datetime"]).dt.day_name()
dataset = df.set_index("Datetime")
dataset.index = pd.to_datetime(dataset.index)
dataset.head(1)

C:\Users\R.Vejay\AppData\Local\Temp\ipykernel_83540\3318638198.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataset["Week"] = pd.to_datetime(df["Datetime"]).dt.week


,AEP_MW,Month,Year,Date,Time,Week,Day
Datetime,,,,,,,
2004-12-31 01:00:00,13478,12,2004,2004-12-31,01:00:00,53,Friday


In [7]:
from sklearn.preprocessing import MinMaxScaler

# Your code here, where you use MinMaxScaler

NewDataSet = dataset.resample('D').mean()
print("Old Dataset ",dataset.shape )
print("New  Dataset ",NewDataSet.shape )

TestData = NewDataSet.tail(100)

Training_Set = NewDataSet.iloc[:,0:1]

Training_Set = Training_Set[:-60]
print("Training Set Shape ", Training_Set.shape)
print("Test Set Shape ", TestData.shape)

Training_Set = Training_Set.values
sc = MinMaxScaler(feature_range=(0, 1))
Train = sc.fit_transform(Training_Set)
X_Train = []
Y_Train = []

# Range should be fromm 60 Values to END
for i in range(60, Train.shape[0]):

    # X_Train 0-59
    X_Train.append(Train[i-60:i])

    # Y Would be 60 th Value based on past 60 Values
    Y_Train.append(Train[i])

# Convert into Numpy Array
X_Train = np.array(X_Train)
Y_Train = np.array(Y_Train)

print(X_Train.shape)
print(Y_Train.shape)

Old Dataset  (121273, 7)
New  Dataset  (5446, 4)
Training Set Shape  (5386, 1)
Test Set Shape  (100, 4)
(5326, 60, 1)
(5326, 1)


In [8]:
X_Train = np.reshape(X_Train, newshape=(X_Train.shape[0], X_Train.shape[1], 1))
X_Train.shape

(5326, 60, 1)

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import LSTM, Dense,Dropout

# Your code here, where you use Sequential, LSTM, and other Keras components

# Your code here, where you use Sequential and other Keras components

regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_Train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

now we are going to fit the created regressor modle with the 4 layers

In [10]:
regressor.fit(X_Train, Y_Train, epochs = 10, batch_size = 16)

Epoch 1/10

333/333 [==============================] - 23s 52ms/step - loss: nan
Epoch 2/10
333/333 [==============================] - 17s 50ms/step - loss: nan
Epoch 3/10
333/333 [==============================] - 18s 55ms/step - loss: nan
Epoch 4/10
333/333 [==============================] - 21s 62ms/step - loss: nan
Epoch 5/10
333/333 [==============================] - 22s 66ms/step - loss: nan
Epoch 6/10
333/333 [==============================] - 22s 67ms/step - loss: nan
Epoch 7/10
333/333 [==============================] - 22s 65ms/step - loss: nan
Epoch 8/10
333/333 [==============================] - 22s 66ms/step - loss: nan
Epoch 9/10
333/333 [==============================] - 22s 67ms/step - loss: nan
Epoch 10/10
333/333 [==============================] - 23s 68ms/step - loss: nan
